In [1]:
from sqlalchemy import create_engine, text

In [2]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase demo"""
    login = 'student'
    password = 'student!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

In [3]:
engine = connection_yandex_cloud_demo(echo=True)

**! Просьба в ячейке ниже указать наименование вашей схемы, чтобы выполняемые запросы не пересекались между собой**

In [4]:
SCHEMA_NAME = 'public'

### Оптимизация с помощью индекса

**Создадим таблицу для экспериментов и заполним её данными**

In [5]:
sql = f'''
DROP TABLE IF EXISTS {SCHEMA_NAME}.ticket_flights_copy;

CREATE TABLE {SCHEMA_NAME}.ticket_flights_copy
 (LIKE bookings.ticket_flights);
 
INSERT INTO {SCHEMA_NAME}.ticket_flights_copy
SELECT * FROM bookings.ticket_flights
'''

engine.execute(sql)

/var/folders/g7/ydw5y9ls4_l8cbw38l0v1p_80000gn/T/ipykernel_12583/2364493803.py:11: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(sql)


2024-02-13 21:03:03,420 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-02-13 21:03:03,422 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:03:03,472 INFO sqlalchemy.engine.Engine select current_schema()
2024-02-13 21:03:03,473 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:03:03,505 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-02-13 21:03:03,506 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:03:03,551 INFO sqlalchemy.engine.Engine 
DROP TABLE IF EXISTS public.ticket_flights_copy;

CREATE TABLE public.ticket_flights_copy
 (LIKE bookings.ticket_flights);
 
INSERT INTO public.ticket_flights_copy
SELECT * FROM bookings.ticket_flights

2024-02-13 21:03:03,552 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:06:40,101 INFO sqlalchemy.engine.Engine COMMIT


**Посмотрим кол-во записей в созданной таблице**

In [6]:
sql = f'''
 SELECT count(*)
   FROM {SCHEMA_NAME}.ticket_flights_copy
'''

engine.execute(sql).fetchall()[0][0]

2024-02-13 21:08:13,013 INFO sqlalchemy.engine.Engine 
 SELECT count(*)
   FROM public.ticket_flights_copy

2024-02-13 21:08:13,017 INFO sqlalchemy.engine.Engine [raw sql] {}


8391852

**Сделаем запрос на поиск данных по перелету с идентификатором 130525**

In [7]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525
'''

engine.execute(text(sql)).fetchall()

2024-02-13 21:08:38,325 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525

2024-02-13 21:08:38,330 INFO sqlalchemy.engine.Engine [generated in 0.00197s] {}


[('Gather  (cost=1000.00..114647.56 rows=70 width=32) (actual time=0.254..594.082 rows=93 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on ticket_flights_copy tfc  (cost=0.00..113640.56 rows=29 width=32) (actual time=0.869..575.173 rows=31 loops=3)',),
 ('        Filter: (flight_id = 130525)',),
 ('        Rows Removed by Filter: 2797253',),
 ('Planning Time: 0.164 ms',),
 ('Execution Time: 594.132 ms',)]

**Попробуем использовать нечеткий поиск, посмотрим как изменлось время выполнения запроса**

In [8]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%130525%'
'''

engine.execute(text(sql)).fetchall()

2024-02-13 21:09:56,956 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE 
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%%130525%%'

2024-02-13 21:09:56,958 INFO sqlalchemy.engine.Engine [generated in 0.00195s] {}


[('Gather  (cost=1000.00..132392.09 rows=2685 width=32) (actual time=0.396..1879.029 rows=93 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on ticket_flights_copy tfc  (cost=0.00..131123.59 rows=1119 width=32) (actual time=4.789..1859.077 rows=31 loops=3)',),
 ("        Filter: ((flight_id)::text ~~ '%130525%'::text)",),
 ('        Rows Removed by Filter: 2797253',),
 ('Planning Time: 0.076 ms',),
 ('Execution Time: 1879.085 ms',)]

**Создадим индекс на поле bookings.ticket_flights_copy.flight_id**

In [9]:
sql = f'''
DROP INDEX IF EXISTS ticket_flights_copy_flight_id_index;

CREATE INDEX ticket_flights_copy_flight_id_index 
          ON {SCHEMA_NAME}.ticket_flights_copy (flight_id);
'''

engine.execute(sql)

2024-02-13 21:11:00,415 INFO sqlalchemy.engine.Engine 
DROP INDEX IF EXISTS ticket_flights_copy_flight_id_index;

CREATE INDEX ticket_flights_copy_flight_id_index 
          ON public.ticket_flights_copy (flight_id);

2024-02-13 21:11:00,417 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:11:41,671 INFO sqlalchemy.engine.Engine COMMIT


**Выполним запросы после создания индекса, сравним время выполнения с предыдущими результатом**

In [10]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525
'''

engine.execute(text(sql)).fetchall()

2024-02-13 21:12:04,206 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525

2024-02-13 21:12:04,208 INFO sqlalchemy.engine.Engine [cached since 205.9s ago] {}


[('Index Scan using ticket_flights_copy_flight_id_index on ticket_flights_copy tfc  (cost=0.43..72.61 rows=70 width=32) (actual time=0.060..0.168 rows=93 loops=1)',),
 ('  Index Cond: (flight_id = 130525)',),
 ('Planning Time: 0.150 ms',),
 ('Execution Time: 0.195 ms',)]

In [20]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%130525%'
'''

engine.execute(text(sql)).fetchall()

2024-02-13 21:19:55,700 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE 
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%%130525%%'

2024-02-13 21:19:55,701 INFO sqlalchemy.engine.Engine [cached since 598.8s ago] {}


[('Gather  (cost=1000.00..132392.09 rows=2685 width=32) (actual time=0.259..1449.321 rows=93 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on ticket_flights_copy tfc  (cost=0.00..131123.59 rows=1119 width=32) (actual time=6.736..1430.012 rows=31 loops=3)',),
 ("        Filter: ((flight_id)::text ~~ '%130525%'::text)",),
 ('        Rows Removed by Filter: 2797253',),
 ('Planning Time: 0.051 ms',),
 ('Execution Time: 1449.367 ms',)]

**Создадим копию таблицы flights и выполним запрос с фильтрацией по двум атрибутам**

In [15]:
sql = f'''
DROP TABLE IF EXISTS {SCHEMA_NAME}.flights_copy;

CREATE TABLE {SCHEMA_NAME}.flights_copy
 (LIKE bookings.flights);
 
INSERT INTO {SCHEMA_NAME}.flights_copy
SELECT * FROM bookings.flights
'''

engine.execute(sql)

2024-02-13 21:15:16,480 INFO sqlalchemy.engine.Engine 
DROP TABLE IF EXISTS public.flights_copy;

CREATE TABLE public.flights_copy
 (LIKE bookings.flights);
 
INSERT INTO public.flights_copy
SELECT * FROM bookings.flights

2024-02-13 21:15:16,482 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:15:17,039 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

2024-02-13 21:17:36,964 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'

2024-02-13 21:17:36,967 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Gather  (cost=1000.00..5639.69 rows=1198 width=63) (actual time=0.252..27.489 rows=1584 loops=1)',),
 ('  Workers Planned: 1',),
 ('  Workers Launched: 1',),
 ('  ->  Parallel Seq Scan on flights_copy fc  (cost=0.00..4519.89 rows=705 width=63) (actual time=10.183..22.270 rows=792 loops=2)',),
 ("        Filter: ((departure_airport = 'DME'::bpchar) AND (arrival_airport = 'LED'::bpchar))",),
 ('        Rows Removed by Filter: 106642',),
 ('Planning Time: 0.224 ms',),
 ('Execution Time: 27.600 ms',)]

**Создадим составной индекс**

In [17]:
sql = f'''
DROP INDEX IF EXISTS flights_copy_airports_index;

CREATE INDEX flights_copy_airports_index 
          ON {SCHEMA_NAME}.flights_copy (departure_airport, arrival_airport);
'''

engine.execute(sql)

2024-02-13 21:18:04,560 INFO sqlalchemy.engine.Engine 
DROP INDEX IF EXISTS flights_copy_airports_index;

CREATE INDEX flights_copy_airports_index 
          ON public.flights_copy (departure_airport, arrival_airport);

2024-02-13 21:18:04,562 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:18:05,005 INFO sqlalchemy.engine.Engine COMMIT


**Выполним этот запрос повторно**

In [18]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

2024-02-13 21:18:25,966 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'

2024-02-13 21:18:25,968 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Scan using flights_copy_airports_index on flights_copy fc  (cost=0.29..995.47 rows=1198 width=63) (actual time=0.055..0.295 rows=1584 loops=1)',),
 ("  Index Cond: ((departure_airport = 'DME'::bpchar) AND (arrival_airport = 'LED'::bpchar))",),
 ('Planning Time: 0.136 ms',),
 ('Execution Time: 0.376 ms',)]

**Выведем только те поля, которые участвуют в индексе**

In [21]:
sql = f'''
EXPLAIN ANALYZE
 SELECT fc.departure_airport
       ,fc.arrival_airport
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

2024-02-13 21:20:26,380 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT fc.departure_airport
       ,fc.arrival_airport
   FROM public.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'

2024-02-13 21:20:26,381 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Only Scan using flights_copy_airports_index on flights_copy fc  (cost=0.29..26.26 rows=1198 width=8) (actual time=0.027..0.150 rows=1584 loops=1)',),
 ("  Index Cond: ((departure_airport = 'DME'::bpchar) AND (arrival_airport = 'LED'::bpchar))",),
 ('  Heap Fetches: 0',),
 ('Planning Time: 0.063 ms',),
 ('Execution Time: 0.222 ms',)]

**Допустим, нам необходимо посмотреть данные за определённую дату о совершенных полётах**

In [22]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE DATE(fc.actual_departure) = '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-02-13 21:21:22,364 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE DATE(fc.actual_departure) = '2017-09-06'

2024-02-13 21:21:22,365 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Gather  (cost=1000.00..5627.29 rows=1074 width=63) (actual time=44.925..47.844 rows=0 loops=1)',),
 ('  Workers Planned: 1',),
 ('  Workers Launched: 1',),
 ('  ->  Parallel Seq Scan on flights_copy fc  (cost=0.00..4519.89 rows=632 width=63) (actual time=42.117..42.118 rows=0 loops=2)',),
 ("        Filter: (date(actual_departure) = '2017-09-06'::date)",),
 ('        Rows Removed by Filter: 107434',),
 ('Planning Time: 0.092 ms',),
 ('Execution Time: 47.881 ms',)]

**Сравним время выполнения фильтрации по равенству и наравенству**

In [23]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE DATE(fc.actual_departure) > '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-02-13 21:21:45,347 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE DATE(fc.actual_departure) > '2017-09-06'

2024-02-13 21:21:45,348 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Seq Scan on flights_copy fc  (cost=0.00..5847.00 rows=71622 width=63) (actual time=41.892..41.892 rows=0 loops=1)',),
 ("  Filter: (date(actual_departure) > '2017-09-06'::date)",),
 ('  Rows Removed by Filter: 214867',),
 ('Planning Time: 0.052 ms',),
 ('Execution Time: 41.925 ms',)]

**Создадим индекс на колонку с датой отправления**

In [24]:
sql = f'''
DROP INDEX IF EXISTS flights_copy_actual_departure_index;

CREATE INDEX flights_copy_actual_departure_index ON {SCHEMA_NAME}.flights_copy (actual_departure);
'''

engine.execute(sql)

2024-02-13 21:23:09,908 INFO sqlalchemy.engine.Engine 
DROP INDEX IF EXISTS flights_copy_actual_departure_index;

CREATE INDEX flights_copy_actual_departure_index ON public.flights_copy (actual_departure);

2024-02-13 21:23:09,910 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:23:10,254 INFO sqlalchemy.engine.Engine COMMIT


**Проверим, ускорился ли наш запрос**

In [25]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.actual_departure = '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-02-13 21:23:17,533 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.actual_departure = '2017-09-06'

2024-02-13 21:23:17,535 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Scan using flights_copy_actual_departure_index on flights_copy fc  (cost=0.42..2.44 rows=1 width=63) (actual time=0.031..0.031 rows=0 loops=1)',),
 ("  Index Cond: (actual_departure = '2017-09-06 00:00:00+03'::timestamp with time zone)",),
 ('Planning Time: 0.213 ms',),
 ('Execution Time: 0.067 ms',)]

**Сравним время выполнения фильтрации по равенству и наравенству на индексе**

In [26]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.actual_departure > '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-02-13 21:23:32,546 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.actual_departure > '2017-09-06'

2024-02-13 21:23:32,547 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Scan using flights_copy_actual_departure_index on flights_copy fc  (cost=0.42..2.44 rows=1 width=63) (actual time=0.006..0.006 rows=0 loops=1)',),
 ("  Index Cond: (actual_departure > '2017-09-06 00:00:00+03'::timestamp with time zone)",),
 ('Planning Time: 0.160 ms',),
 ('Execution Time: 0.033 ms',)]

**UNION vs UNION ALL**

In [27]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

2024-02-13 21:24:19,877 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION
 SELECT * 
   FROM public.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'

2024-02-13 21:24:19,879 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Unique  (cost=43364.47..49273.31 rows=214867 width=170) (actual time=106.769..176.518 rows=214867 loops=1)',),
 ('  ->  Sort  (cost=43364.47..43901.63 rows=214867 width=170) (actual time=106.767..131.787 rows=214867 loops=1)',),
 ('        Sort Key: fc.flight_id, fc.flight_no, fc.scheduled_departure, fc.scheduled_arrival, fc.departure_airport, fc.arrival_airport, fc.status, fc.aircraft_code, fc.actual_departure, fc.actual_arrival',),
 ('        Sort Method: external merge  Disk: 17112kB',),
 ('        ->  Append  (cost=0.00..13842.68 rows=214867 width=170) (actual time=0.019..65.052 rows=214867 loops=1)',),
 ('              ->  Seq Scan on flights_copy fc  (cost=0.00..5309.84 rows=75337 width=63) (actual time=0.018..28.056 rows=75402 loops=1)',),
 ("                    Filter: (scheduled_departure < '2017-01-01 00:00:00+03'::timestamp with time zone)",),
 ('                    Rows Removed by Filter: 139465',),
 ('              ->  Seq Scan on flights_copy fc2  (cost=0.00..5309.84 r

In [28]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION ALL
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

2024-02-13 21:24:25,385 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE 
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION ALL
 SELECT * 
   FROM public.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'

2024-02-13 21:24:25,386 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Append  (cost=0.00..13842.68 rows=214867 width=63) (actual time=0.019..68.963 rows=214867 loops=1)',),
 ('  ->  Seq Scan on flights_copy fc  (cost=0.00..5309.84 rows=75337 width=63) (actual time=0.019..18.690 rows=75402 loops=1)',),
 ("        Filter: (scheduled_departure < '2017-01-01 00:00:00+03'::timestamp with time zone)",),
 ('        Rows Removed by Filter: 139465',),
 ('  ->  Seq Scan on flights_copy fc2  (cost=0.00..5309.84 rows=139530 width=63) (actual time=0.008..33.001 rows=139465 loops=1)',),
 ("        Filter: (scheduled_departure >= '2017-01-01 00:00:00+03'::timestamp with time zone)",),
 ('        Rows Removed by Filter: 75402',),
 ('Planning Time: 0.129 ms',),
 ('Execution Time: 77.761 ms',)]

**Временные таблицы**

In [33]:
sql = f'''
EXPLAIN ANALYSE
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
FROM {SCHEMA_NAME}.flights_copy fc
JOIN {SCHEMA_NAME}.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id 
'''

engine.execute(sql).fetchall()

2024-02-13 21:40:14,432 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYSE
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
FROM public.flights_copy fc
JOIN public.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id 

2024-02-13 21:40:14,435 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Merge Join  (cost=0.89..315098.96 rows=8391852 width=91) (actual time=0.019..7101.362 rows=8391852 loops=1)',),
 ('  Merge Cond: (fc.flight_id = tfc.flight_id)',),
 ('  ->  Index Scan using flights_copy_flight_id_idx on flights_copy fc  (cost=0.42..6440.43 rows=214867 width=63) (actual time=0.009..48.364 rows=214867 loops=1)',),
 ('  ->  Index Scan using ticket_flights_copy_flight_id_index on ticket_flights_copy tfc  (cost=0.43..203223.22 rows=8391852 width=32) (actual time=0.006..5520.391 rows=8391852 loops=1)',),
 ('Planning Time: 0.253 ms',),
 ('Execution Time: 7410.362 ms',)]

**Создаём временную таблицу**

In [30]:
sql = f'''
CREATE TEMPORARY TABLE ticket_flights_materialized_{SCHEMA_NAME} AS 
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
  FROM {SCHEMA_NAME}.flights_copy fc
  JOIN {SCHEMA_NAME}.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id;
'''

engine.execute(sql)

2024-02-13 21:35:45,588 INFO sqlalchemy.engine.Engine 
CREATE TEMPORARY TABLE ticket_flights_materialized_public AS 
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
  FROM public.flights_copy fc
  JOIN public.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id;

2024-02-13 21:35:45,590 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 21:36:55,793 INFO sqlalchemy.engine.Engine COMMIT


**Сделаем запрос к веременной таблице. Как изменилось время выполнения?**

In [31]:
sql = f'''
EXPLAIN ANALYSE
 SELECT tfm.*
   FROM ticket_flights_materialized_{SCHEMA_NAME} tfm;
'''

engine.execute(sql).fetchall()

2024-02-13 21:37:02,785 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYSE
 SELECT tfm.*
   FROM ticket_flights_materialized_public tfm;

2024-02-13 21:37:02,786 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Seq Scan on ticket_flights_materialized_public tfm  (cost=0.00..172613.70 rows=3561870 width=280) (actual time=931.778..52876.701 rows=8391852 loops=1)',),
 ('Planning Time: 0.094 ms',),
 ('Execution Time: 62897.597 ms',)]